In [1]:
import time
import pymongo
import re
import json
import requests
import pandas as pd

from tqdm import tqdm
from pprint import pprint
from pathlib import Path
from bs4 import BeautifulSoup

#import local libraries
import lib_sepinpho as sep
import lib_fileops as io

##### INIT GLOBAL VARIABLES#####

#init Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#connect to database
db = client.visualizing_sep

In [ ]:
#export list of all articles with domain_tags and primary_domain
# commented out to not accidently run again

# collection_to_update = db.sep_entries
# all_domains = pd.DataFrame(collection_to_update.find({},
#                                               {'title':1, 'page_url', 'author': 1, 'primary_domain':1, 'domain_tags':1}))
# all_domains.to_csv('domains.csv')

In [ ]:
#import updated domain data
# commented out to not accidently run again

collection_to_update = db.sep_entries


# updated_domains = pd.read_csv('domains.csv')

# for index,row in tqdm(updated_domains.iterrows()):
#     title = row['title']
#     domain_tags = row['domain_tags']
#     primary_domain = row['primary_domain']
#     # update function to include page_url
#     sep.update_domain_info(title,domain_tags, primary_domain, collection_to_update)

In [52]:
#get distinct list of primary_domains
db.sep_entries.distinct('primary_domain')

['Aesthetics and Philosophy of Art',
 'African and African-American Philosophy',
 'Arabic and Islamic Philosophy',
 'Chinese Philosophy',
 'Continental Philosophy',
 'Epistemology',
 'Ethics',
 'Feminist Philosophy',
 'Indian Philosophy',
 'Japanese Philosophy',
 'Latin American and Iberian Philosophy',
 'Logic',
 'Metaphysics',
 'Philosophy of Biology',
 'Philosophy of Computer Science',
 'Philosophy of Economics',
 'Philosophy of Language',
 'Philosophy of Law',
 'Philosophy of Mathematics',
 'Philosophy of Mind',
 'Philosophy of Physics',
 'Philosophy of Religion',
 'Philosophy of Science',
 'Social and Political Philosophy',
 'Thinker']

In [58]:
#extract unique list of domain tags
domain_tags = list(db.sep_entries.find( filter={},
                                     projection={'domain_tags':1, '_id':0}))
domain_tags_individual = []

for tag in domain_tags:
    semisplit_tags = tag['domain_tags'].split(';')
    for semisplit_tag in semisplit_tags:
        commasplit_tags = semisplit_tag.split(',')
        for commasplit_tag in commasplit_tags:
            if commasplit_tag != '':
                domain_tags_individual.append(commasplit_tag.strip())

individual_tags = sorted(set(domain_tags_individual))
pprint(individual_tags)

['Aesthetics and Philosophy of Art',
 'African and African-American Philosophy',
 'Arabic and Islamic Philosophy',
 'Chinese Philosophy',
 'Continental Philosophy',
 'Epistemology',
 'Ethics',
 'Feminist Philosophy',
 'Indian Philosophy',
 'Japanese Philosophy',
 'Jewish Philosophy',
 'Latin American and Iberian Philosophy',
 'Logic',
 'Metaphysics',
 'Philosophy of Biology',
 'Philosophy of Computer Science',
 'Philosophy of Economics',
 'Philosophy of Language',
 'Philosophy of Law',
 'Philosophy of Mathematics',
 'Philosophy of Mind',
 'Philosophy of Physics',
 'Philosophy of Religion',
 'Philosophy of Science',
 'Social and Political Philosophy',
 'Thinker']


In [8]:
#update db with new domain names
collection_to_update = db.sep_entries

search_filter = 'Latin American Philosophy'
replace_with = 'Latin American and Iberian Philosophy'

results = list(collection_to_update.find({'primary_domain': re.compile(search_filter)}, 
                                         {'page_url': 1, 'title':1, 'primary_domain':1}))

for result in results:
    print(result['primary_domain'])
    new_domain_tags = result['primary_domain'].replace(search_filter, replace_with)
    print(new_domain_tags)
    collection_to_update.update_one({'page_url': result['page_url']}, { '$set': {'primary_domain': new_domain_tags}})

new_results = list(collection_to_update.find({'primary_domain': re.compile(search_filter)}, 
                                         {'page_url': 1, 'title':1, 'primary_domain':1}))
pprint(new_results)

[]


In [ ]:
# create text search on preamble_text and main_text
# completed 10.1.2020

collection_to_update = db.sep_entries

collection_to_update.create_index([('preamble_text','text'), ('main_text','text')], name='preamble_textindex')
collection_to_update.index_information()

In [ ]:
#Prior Updates
# African American Philosophy: Completed 10.2.2020
# text_filter = 'African African-American'

#########################################################
# Completed 11.18.2020

# Jewish Philosophy: 
# text_filter = ' "\"Jewish Philosophy\ "\"Hebrew Philosophy\  '

# Aesthetics and Philosophy of Art
# text_filter = ' "\"Jewish Philosophy\ "\"Hebrew Philosophy\  '

# Arabic and Islamic Philosophy
# text_filter = '  "\"Arabic and Islamic Philosophy\ "\"Islamic Philosophy\ "\"Arabic Philosophy\ '

# Chinese Philosophy
# text_filter = '  "\"Chinese Philosophy\ Buddhism Daoism '

# Epistemology
# text_filter = ' Epistemology "\"Theory of Knowedge Truth '

#Feminist Philosophy
# text_filter = ' Feminism "\"Feminist Philosophy\ '

#Indian Philosophy
# text_filter = ' "\"Indian Philosophy\ '

#Japanese Philosophy
# text_filter = ' "\"Japanese Philosophy\ '

#Latin American
# text_filter = ' "\"Latin American Philosophy\ "\"Iberian Philosophy "\"Spanish Philosophy '

#Logic 
# text_filter = ' Logic Inference Entailment'

#Metaphysics
# text_filter = ' Logic Inference Entailment'

#philosopy of biology
# text_filter = ' "\"Philosophy of Biology\ Biology Evolution Genetics Biological Conscoiusness Human Neuroscience'

#philosophy of computer science
# text_filter = ' "\"Computer Science\ "\"Information Technology\ Computation '

#philosophy of economics
# text_filter = ' "\"Philosophy of Economics\ "\"Game Theory\ Economic Markets Capitalism Wealth "\"Decision Theory\ '

#philosophy of language
# text_filter = ' "\"Philosophy of Language\ Linguistics Semantics Pragmatics '

#philosophy of law
# text_filter = ' "\"Philosophy of Law\ Legal Law Testimony'

#philosophy of mathematics
# text_filter = ' "\"Philosophy of Mat\ Legal Law Testimony'

# Philosophy of Mind'
# text_filter = ' "\"Philosophy of Mind\ Mind Consciousness Cognition Emotion Brain Mental Perception Intention'

#  'Philosophy of Physics',
# text_filter = ' "\"Philosophy of Physics\ Physics Relativity Quantum''

#  'Philosophy of Religion',
# text_filter = ' ' "\"Philosophy of Religion\ Theology God Divine Religion''

#  'Philosophy of Science',
# text_filter = '"\"Philosophy of Science\ Scientific  '

#  'Social and Political Philosophy',
# text_filter = ' '
#  'Thinker'
#########################################################

In [49]:
#generate dataframes of domain search results, and then export them to csv for content updates

collection_to_update = db.sep_entries

text_filter = ' "\"Political Philosophy \ Politics "\"Social Philosophy\ Sociology Justice Government Rights  '
csv_file = 'politics.csv'
text_results = pd.DataFrame(collection_to_update.find(
                                         {'$text':{'$search': text_filter }},
                                         {'score': { '$meta': 'textScore'}, 
                                            'title':1, 
                                            'page_url':1,
                                            'primary_domain':1, 
                                            'domain_tags':1}
                                         ))

text_results.head(10)                                         
text_score_max =  text_results['score'].max()

text_results_above1 = text_results[text_results.score > text_score_max*0.5]

text_results_above1.to_csv(csv_file)


In [50]:
#update domains from csv 
collection_to_update = db.sep_entries
domains_to_update = pd.read_csv('politics.csv')

for index,row in tqdm(domains_to_update.iterrows()):
    title = row['title']
    domain_tags = row['domain_tags']
    primary_domain = row['primary_domain']
    page_url = row['page_url']

    sep.update_domain_info(page_url,domain_tags, primary_domain, collection_to_update)

 of Religion', 'primary_domain': 'Thinker'}]
acknowledged: /entries/friedrich-lange/
 True
[{'_id': ObjectId('5f1bb33b896f82cdbda61114'), 'title': 'Friedrich Albert Lange', 'domain_tags': 'Thinker,Metaphysics', 'primary_domain': 'Thinker'}]
acknowledged: /entries/wilhelm-humboldt/
 True
[{'_id': ObjectId('5f1bb9e8896f82cdbda6152e'), 'title': 'Wilhelm von Humboldt', 'domain_tags': 'Thinker, Philosophy of Language', 'primary_domain': 'Thinker'}]
acknowledged: /entries/disability-justice/
 True
[{'_id': ObjectId('5f1bb1dc896f82cdbda6104c'), 'title': 'Disability and Justice', 'domain_tags': 'Social and Political Philosophy,Ethics', 'primary_domain': 'Social and Political Philosophy'}]
acknowledged: /entries/lawphil-nature/
 True
[{'_id': ObjectId('5f1bb4c7896f82cdbda6120f'), 'title': 'The Nature of Law', 'domain_tags': 'Philosophy of Law', 'primary_domain': 'Philosophy of Law'}]
acknowledged: /entries/justice/
 True
[{'_id': ObjectId('5f1bb481896f82cdbda611da'), 'title': 'Justice', 'domain